# Projet M2 MIAS : Risque de réidentification (Février 2026)

**Auteur :** [Votre Nom]
**Superviseur :** Pr Emmanuel Chazard

## 1. Objectif
Ce notebook analyse les risques de réidentification sur des bases de données de santé synthétiques, selon les trois critères du G29 (CNIL) :
1.  **Individualisation** : Capacité à isoler un individu.
2.  **Inférence** : Capacité à déduire une information sensible inconnue.
3.  **Corrélation** : Capacité à lier des données (conservation de la structure statistique).

Nous comparons l'efficacité de l'anonymisation par mélange (shuffling) sur des fichiers directs et échantillonnés.

---

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob

# Configuration graphique
plt.style.use('seaborn-v0_8')

## 2. Chargement des Données et de la Vérité Terrain

Nous chargeons :
1.  `connaissances_externes.txt` : La base de connaissance de l'attaquant.
2.  `out_direct_0.txt` : La base originale non mélangée, qui servira de **Vérité Terrain (Ground Truth)** pour vérifier si les inférences de l'attaquant sont correctes.
3.  Les fichiers variants (`out_direct_XX`, `out_sample_XX`) qui sont les bases publiées/attaquées.

In [ ]:
DATA_DIR = "projets_donnees"

def load_data(directory):
    data = {}
    
    # 1. Chargement de la référence (Connaissances de l'attaquant)
    ref_path = os.path.join(directory, "connaissances_externes.txt")
    if os.path.exists(ref_path):
        data['reference'] = pd.read_csv(ref_path, sep=None, engine='python')
        print(f"Base référence chargée : {len(data['reference'])} lignes")
    else:
        raise FileNotFoundError(f"Fichier manquant : {ref_path}")

    # 2. Chargement des fichiers publiés (Variants)
    files = glob.glob(os.path.join(directory, "out_*.txt"))
    if not files: print("Attention : Aucun fichier 'out_*.txt' trouvé.")

    for filepath in files:
        filename = os.path.basename(filepath)
        name_key = filename.replace('.txt', '') # ex: out_direct_10
        try:
            data[name_key] = pd.read_csv(filepath, sep=None, engine='python')
            print(f" -> Chargé : {name_key}")
        except Exception as e:
            print(f"Erreur {filename} : {e}")
    return data

# Exécution du chargement
datasets = load_data(DATA_DIR)
df_reference = datasets['reference']

# Définition de la Vérité Terrain pour vérifier l'Inférence
# On utilise le fichier sans mélange (0%) pour connaitre les vraies maladies des patients
if 'out_direct_0' in datasets:
    df_truth = datasets['out_direct_0'].set_index('id_sejour')
elif 'out_sample_0' in datasets:
    df_truth = datasets['out_sample_0'].set_index('id_sejour')
else:
    df_truth = None
    print("Attention : Pas de fichier '_0.txt' pour la vérité terrain. L'inférence sera à 0.")

## 3. Méthodologie et Indicateurs

### 3.1 Risque d'Individualisation (Taux de Réidentification)
**Technique : Attaque par jointure sur les uniques**
Nous simulons un attaquant qui cherche des individus uniques dans la base publiée.
1.  Identification des lignes uniques sur les quasi-identifiants (QI) dans le fichier publié.
2.  Jointure avec la base de connaissance.
3.  Vérification via l'`id_sejour` (clé secrète). Si l'ID correspond, l'individu est réidentifié.

### 3.2 Risque d'Inférence (Précision de l'attribut)
**Technique : Comparaison avec la Vérité Terrain**
L'attaquant devine la valeur d'une variable sensible (ex: `liste_diag`) à partir de la ligne qu'il a trouvée.
1.  L'attaquant lit la valeur `liste_diag` dans le fichier publié mélangé.
2.  Nous comparons cette valeur avec la **vraie** valeur issue du fichier `out_direct_0` (Vérité Terrain).
3.  Si les valeurs sont identiques, l'inférence est réussie (le mélange n'a pas protégé cette info).

### 3.3 Risque de Corrélation (Conservation de structure)
**Technique : Corrélation de Spearman**
Mesure si les liens statistiques entre les variables (ex: Âge vs Pathologie) sont préservés.
1.  Calcul des matrices de corrélation de Spearman pour la référence et le fichier publié.
2.  Calcul de la différence moyenne. Si la différence est faible, le risque est élevé (structure conservée).

In [ ]:
def calculate_correlation_risk(df_ref, df_anon):
    """Score de 0 à 100% indiquant la conservation de la structure des corrélations."""
    common_cols = [c for c in df_ref.columns if c in df_anon.columns and c != 'id_sejour']
    if not common_cols: return 0.0
    
    # Encodage factoriel pour les variables catégorielles
    def get_encoded_corr(df, cols):
        temp = pd.DataFrame()
        for c in cols:
            if df[c].dtype == 'object': temp[c] = pd.factorize(df[c])[0]
            else: temp[c] = df[c]
        return temp.corr(method='spearman')

    corr_ref = get_encoded_corr(df_ref, common_cols).fillna(0)
    corr_anon = get_encoded_corr(df_anon, common_cols).fillna(0)
    
    # Différence moyenne
    mean_diff = (corr_ref - corr_anon).abs().mean().mean()
    return max(0, (1 - mean_diff * 2)) * 100

def calculate_risk_metrics(df_connaissance, df_publie, quasi_identifiers, sensitive_col, df_truth_ref=None):
    """Calcule Individualisation, Inférence et Corrélation."""
    # 1. Corrélation (Globale)
    risk_corr = calculate_correlation_risk(df_connaissance, df_publie)

    # Vérification présence colonnes QI
    if any(c not in df_publie.columns for c in quasi_identifiers): 
        return 0.0, 0.0, risk_corr

    # 2. Individualisation
    counts = df_publie.groupby(quasi_identifiers).size()
    unique_combs = counts[counts == 1].index
    
    if len(unique_combs) == 0: return 0.0, 0.0, risk_corr
    
    # Filtrage des uniques et Jointure (Attaque)
    df_pub_unique = df_publie.set_index(quasi_identifiers)
    df_pub_unique = df_pub_unique[df_pub_unique.index.isin(unique_combs)].reset_index()
    
    merged = pd.merge(
        df_pub_unique, df_connaissance, 
        on=quasi_identifiers, how='inner', suffixes=('_pub', '_know')
    )
    
    if len(merged) == 0: return 0.0, 0.0, risk_corr
        
    reid_success = merged[merged['id_sejour_pub'] == merged['id_sejour_know']]
    risk_reid = len(reid_success) / len(df_publie) * 100
    
    # 3. Inférence (Avec Vérité Terrain)
    risk_inference = 0.0
    
    # On vérifie si la colonne sensible existe dans le fichier publié
    if sensitive_col in df_publie.columns and df_truth_ref is not None:
        if sensitive_col in df_truth_ref.columns:
            # Nom de la colonne dans le merged (si conflit de nom, pandas a mis _pub, sinon nom original)
            col_pub_name = f"{sensitive_col}_pub" if f"{sensitive_col}_pub" in merged.columns else sensitive_col
            
            # On compare : Valeur vue par l'attaquant VS Vraie valeur (via l'ID de la personne connue)
            # L'ID de référence est 'id_sejour_know' (la personne que l'attaquant pense avoir trouvée)
            try:
                ids_suspects = merged['id_sejour_know']
                # On ne garde que les IDs présents dans la vérité terrain
                valid_mask = ids_suspects.isin(df_truth_ref.index)
                
                if valid_mask.any():
                    subset_merged = merged[valid_mask]
                    # Valeurs réelles pour ces patients
                    true_values = df_truth_ref.loc[subset_merged['id_sejour_know']][sensitive_col].values
                    # Valeurs observées dans le fichier mélangé
                    observed_values = subset_merged[col_pub_name].values
                    
                    # Succès si égalité
                    inf_success_count = np.sum(true_values == observed_values)
                    risk_inference = inf_success_count / len(merged) * 100
            except KeyError:
                risk_inference = 0.0

    return risk_reid, risk_inference, risk_corr

## 4. Exécution des Scénarios

Nous utilisons les scénarios définis, en ciblant la variable **`liste_diag`** pour mesurer le risque d'inférence (car `diabete` est déjà une variable connue dans le scénario Fort).

In [ ]:
scenarios = {
    '1_Faible': ["age10","sexe","entree_mode","sortie_mode","entree_date_y","sortie_date_y"],
    '2_Moyen': ["age5","sexe","entree_mode","sortie_mode","entree_date_ym","sortie_date_ym","specialite","chirurgie"],
    '3_Fort': ["age","sexe","entree_mode","sortie_mode","entree_date_ymd","sortie_date_ymd","specialite","chirurgie","diabete","insuffisance_renale","demence"]
}

# Variable cible pour l'inférence (Inconnue de l'attaquant dans les scénarios)
TARGET_INFERENCE = 'liste_diag'

results = []

for name, df_var in datasets.items():
    if name == 'reference': continue
    
    # Extraction du niveau de mélange
    parts = name.split('_')
    if len(parts) >= 3:
        file_type = 'Direct' if 'direct' in name else 'Sample'
        try: shuffle_level = int(parts[-1])
        except: shuffle_level = 0
    else: continue

    for scen_name, cols in scenarios.items():
        # Calcul des risques
        r_reid, r_inf, r_corr = calculate_risk_metrics(
            df_reference, 
            df_var, 
            cols, 
            sensitive_col=TARGET_INFERENCE,
            df_truth_ref=df_truth
        )
        
        results.append({
            'Type': file_type,
            'Mélange (%)': shuffle_level,
            'Scénario': scen_name,
            'Individualisation (%)': r_reid,
            'Inférence (%)': r_inf,
            'Corrélation (%)': r_corr
        })

df_res = pd.DataFrame(results)
print("Calculs terminés.")
display(df_res.head())

## 5. Visualisation des Résultats

In [ ]:
def plot_all_metrics(df_res):
    if df_res.empty:
        print("Pas de données à afficher.")
        return

    fig, axes = plt.subplots(1, 3, figsize=(22, 6))
    
    # 1. Individualisation
    sns.lineplot(data=df_res, x='Mélange (%)', y='Individualisation (%)', 
                 hue='Scénario', style='Type', markers=True, ax=axes[0])
    axes[0].set_title("Risque d'Individualisation (Réidentification)")
    axes[0].set_ylim(-5, 105)

    # 2. Inférence
    sns.lineplot(data=df_res, x='Mélange (%)', y='Inférence (%)', 
                 hue='Scénario', style='Type', markers=True, ax=axes[1])
    axes[1].set_title(f"Risque d'Inférence (Cible: {TARGET_INFERENCE})")
    axes[1].set_ylim(-5, 105)

    # 3. Corrélation
    # Filtre sur un seul scénario pour lisibilité (métrique globale)
    subset = df_res[df_res['Scénario'] == list(scenarios.keys())[0]]
    sns.lineplot(data=subset, x='Mélange (%)', y='Corrélation (%)', 
                 style='Type', markers=True, ax=axes[2], color='green')
    axes[2].set_title("Risque de Corrélation (Conservation Structure)")
    axes[2].set_ylim(-5, 105)

    plt.tight_layout()
    plt.show()

plot_all_metrics(df_res)